# TP1 - EJ4 - Agrupamiento(Clustering)

## Integrantes:


*   Mejia Alan Roberto - 91161

## Objetivos

El objetivo de este ejercicio es buscar los posibles agrupamientos de los datos en funcion de algun criterio.

Los datos a utilizar contiene informacion de algunas tracks(canciones) de Spotify.

# Importaciones

## Instalaciones

In [4]:
#

## Bibliotecas

In [1]:
#Librerías para manejo de datos
import pandas as pd
import numpy as np

#Librerías para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

#Otras librerías
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Bibliotecas con modelos

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.cluster import KMeans

## Datasets

In [3]:
df = pd.read_csv('playlist_spotify.csv')


# Analisis exploratorio

En esta seccion se realiza un analisis necesario para comprender el dominio del problema.

## Forma del dataset

In [6]:
df.shape

(750, 13)

In [7]:
(registros, variables) = df.shape
print("Cantidad de registros: " + str(registros))
print("Cantidad de variables: " + str(variables))

Cantidad de registros: 750
Cantidad de variables: 13


## Variables

In [8]:
variables = df.columns.to_list()
variables

['acousticness',
 'danceability',
 'duration',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'time_signature',
 'valence']

## Datos nulos

In [9]:
(df.isnull().sum() / len(df)) * 100

,0
acousticness,0.0
danceability,0.0
duration,0.0
energy,0.0
instrumentalness,0.0
key,0.0
liveness,0.0
loudness,0.0
mode,0.0
speechiness,0.0


Observaciones: No existen datos nulos

## Tipos de datos de las variables del dataset

In [10]:
tipos = df.dtypes
print(tipos)

acousticness        float64
danceability        float64
duration              int64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object


## Inspeccion de registros

In [11]:
df.head()

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.713,0.514,100125,0.521,0.816000,8,0.1120,-14.835,0,0.0444,119.879,4,0.143
1,0.192,0.714,207019,0.614,0.000000,4,0.2630,-6.935,1,0.0319,123.969,4,0.582
2,0.333,0.630,216200,0.455,0.000004,5,0.1270,-9.290,1,0.0292,139.931,4,0.199
3,0.601,0.810,136413,0.221,0.210000,5,0.1840,-11.005,1,0.0429,109.960,4,0.798
4,0.883,0.465,181440,0.459,0.000173,6,0.0692,-8.137,0,0.0351,90.807,4,0.288


In [12]:
df.tail()

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
745,0.000175,0.374,333827,0.943,0.000156,6,0.1250,-4.108,0,0.0556,112.084,4,0.338
746,0.001970,0.487,213000,0.867,0.006020,10,0.0968,-3.293,0,0.0543,160.048,4,0.403
747,0.916000,0.605,125867,0.314,0.000000,0,0.3590,-7.631,1,0.0327,138.148,4,0.836
748,0.168000,0.700,249493,0.823,0.000028,3,0.1220,-6.892,1,0.0373,144.060,4,0.745
749,0.015500,0.477,276720,0.776,0.003140,1,0.2030,-5.056,1,0.0349,131.004,4,0.429


## Lista variables categoricas

> ToDo: Revisar que variables podrian ser categoricas

In [13]:
variables_categoricas = []

## Lista variables Numericas

In [14]:
variables_numericas = [variable for variable in variables if variable not in variables_categoricas]
variables_numericas

['acousticness',
 'danceability',
 'duration',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'speechiness',
 'tempo',
 'time_signature',
 'valence']

## Estadisticos de variables numericas

In [15]:
df.describe()

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,0.357394,0.596439,220112.733333,0.594188,0.100245,4.829333,0.203376,-8.509339,0.741333,0.098966,120.405761,3.902667,0.497321
std,0.338405,0.172036,65587.690483,0.253301,0.259921,3.636001,0.177609,5.039488,0.438194,0.104715,28.378116,0.400091,0.239615
min,0.000001,0.107000,33840.000000,0.009250,0.000000,0.000000,0.024000,-29.601000,0.000000,0.023400,55.747000,1.000000,0.033200
25%,0.037150,0.480000,185490.250000,0.423250,0.000000,1.000000,0.094550,-10.173500,0.000000,0.035900,98.998000,4.000000,0.297000
50%,0.244500,0.606000,215108.500000,0.631500,0.000010,5.000000,0.129000,-7.270000,1.000000,0.048750,120.104500,4.000000,0.483000
75%,0.678500,0.715750,244236.750000,0.804750,0.002245,8.000000,0.264750,-5.097750,1.000000,0.113000,138.074750,4.000000,0.684500
max,0.994000,0.986000,675360.000000,0.995000,0.967000,11.000000,0.979000,-0.533000,1.000000,0.721000,204.162000,5.000000,0.975000


## Cantidad de Nulos

In [16]:
df.isnull().sum()

,0
acousticness,0
danceability,0
duration,0
energy,0
instrumentalness,0
key,0
liveness,0
loudness,0
mode,0
speechiness,0


No existen datos nulos. Es un dataset limpios de ellos.

# Entrenamiento y Agrupamiento

## Preprocesamiento

Dado que es importante el tema de las distancia. Se pasa a escalar las variables.

## Metodo de Elbow

## Silhouette

## Hopkins: Tendencia al clustering